In [4]:
import numpy as np
from google.colab import files
uploaded = files.upload()  # Selecciona el archivo .npy

snapshots_matrix = np.load('snapshots_matrix (1).npy')
reduced_basis = np.load('reduced_basis (1).npy')
training_set = np.load('training_set (1).npy')
testing_set = np.load('testing_set (1).npy')
result_matrix = np.load('result_matrix (1).npy')
salidas=np.load('salidas (1).npy')
salidas_esperadas=np.load('salidas_esperadas (1).npy')

Saving result_matrix (1).npy to result_matrix (1) (1).npy
Saving salidas (1).npy to salidas (1) (1).npy
Saving salidas_esperadas (1).npy to salidas_esperadas (1) (1).npy
Saving snapshots_matrix (1).npy to snapshots_matrix (1) (1).npy
Saving testing_set (1).npy to testing_set (1) (1).npy
Saving training_set (1).npy to training_set (1) (1).npy
Saving reduced_basis (1).npy to reduced_basis (1) (1).npy


In [5]:
import matplotlib.pyplot as plt
import torch                     # PyTorch: librería principal
import torch.nn as nn            # Para definir la red neuronal
import torch.optim as optim      # Para los optimizadores (SGD, Adam, etc.)

In [36]:
class RED_NEURONAL:
    def __init__(self,
                 n_entrada,
                 m_salida,
                 training_set,
                 salidas,
                 capas_ocultas=2,
                 neuronas_por_capa=32,
                 funcion_costo= None,
                 lr=0.002,
                 tol=1e-7,
                 no_improvement_limit=200,
                 min_delta = 1e-7,
                 max_epochs=10000):

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Usando dispositivo:", self.device)

        self.n = n_entrada
        self.m = m_salida
        self.capas = capas_ocultas
        self.neuronas = neuronas_por_capa
        self.lr = lr
        self.max_epochs = max_epochs
        self.tol = tol
        self.no_improvement_limit = no_improvement_limit
        self.min_delta = min_delta

        # Preparación de datos
        self.X = torch.tensor(training_set, dtype=torch.float32).to(self.device)
        Y_np = np.stack(salidas)
        self.Y = torch.tensor(Y_np, dtype=torch.float32).to(self.device)

        print("Forma de X:", self.X.shape)
        print("Forma de Y:", self.Y.shape)

        # Red neuronal
        self.model = self._construir_modelo().to(self.device)

        # Función de costo
        if funcion_costo is None:
            self.loss_fn = lambda y_pred, y_true: torch.mean((y_pred - y_true) ** 2)  # MSE por defecto
        else:
            self.loss_fn = funcion_costo


        # Optimizador
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)

    def _construir_modelo(self):
        capas = []
        capas.append(nn.Linear(self.n, self.neuronas))
        capas.append(nn.Tanh())
        for _ in range(self.capas - 1):
            capas.append(nn.Linear(self.neuronas, self.neuronas))
            capas.append(nn.Tanh())
        capas.append(nn.Linear(self.neuronas, self.m))
        return nn.Sequential(*capas)

    def entrenar(self):
        best_loss = float("inf")
        best_model_state = None
        epochs_no_improve = 0

        for epoch in range(self.max_epochs):
            self.model.train()
            y_pred = self.model(self.X)
            loss = self.loss_fn(y_pred, self.Y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            # Guardar el mejor modelo
            if loss.item() < best_loss - self.min_delta:
                best_loss = loss.item()
                best_model_state = self.model.state_dict()  # Guardar estado
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            # Condición de parada temprana
            if loss.item() < self.tol or epochs_no_improve >= self.no_improvement_limit:
                print(f"Parada temprana en la época {epoch}, pérdida: {loss.item():.8f}")
                break

            if epoch % 200 == 0:
                print(f"Época {epoch}, Pérdida: {loss.item():.8f}")

        # Restaurar el mejor modelo
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)
            print(f"Modelo restaurado a la mejor pérdida: {best_loss:.8f}")


    def predecir(self, entrada):
        self.model.eval()
        entrada_tensor = torch.tensor(entrada, dtype=torch.float32).to(self.device)
        if entrada_tensor.ndim == 1:
            entrada_tensor = entrada_tensor.unsqueeze(0)
        with torch.no_grad():
            prediccion = self.model(entrada_tensor)
        return prediccion.cpu().numpy()


In [10]:
def custom_loss_factory(reduced_basis_tensor):
    def loss(y_pred, y_true):
        recon_pred = torch.matmul(y_pred, reduced_basis_tensor.T)
        recon_true = torch.matmul(y_true, reduced_basis_tensor.T)
        return torch.mean((recon_pred - recon_true) ** 2) * 100
    return loss


# Con el ejemplo de rigidez fija

In [11]:
# Crear el tensor de base reducida
reduced_basis_tensor = torch.tensor(reduced_basis, dtype=torch.float32).to("cuda" if torch.cuda.is_available() else "cpu")

# Crear la función de pérdida personalizada ya con la base reducida incluida
loss_func = custom_loss_factory(reduced_basis_tensor)

In [ ]:
# Instanciar la red con esa función
red1 = RED_NEURONAL(
    n_entrada=3,
    m_salida=3,
    training_set=training_set,
    salidas=salidas,
    funcion_costo=loss_func,
    max_epochs=8000,
    capas_ocultas=2,
    neuronas_por_capa=32

)

Usando dispositivo: cpu
Forma de X: torch.Size([100, 3])
Forma de Y: torch.Size([100, 3])


In [ ]:
red1.entrenar()

Época 0, Pérdida: 14.896412
Época 200, Pérdida: 0.017380
Época 400, Pérdida: 0.007385
Época 600, Pérdida: 0.004384
Época 800, Pérdida: 0.003049
Época 1000, Pérdida: 0.002283
Época 1200, Pérdida: 0.001776
Época 1400, Pérdida: 0.001416
Época 1600, Pérdida: 0.001150
Época 1800, Pérdida: 0.000946
Época 2000, Pérdida: 0.000800
Época 2200, Pérdida: 0.000703
Época 2400, Pérdida: 0.000586
Época 2600, Pérdida: 0.000515
Época 2800, Pérdida: 0.000475
Época 3000, Pérdida: 0.000395
Época 3200, Pérdida: 0.000356
Época 3400, Pérdida: 0.000306
Época 3600, Pérdida: 0.000278
Época 3800, Pérdida: 0.000240
Época 4000, Pérdida: 0.000221
Época 4200, Pérdida: 0.039652
Época 4400, Pérdida: 0.000255
Época 4600, Pérdida: 0.000221
Época 4800, Pérdida: 0.000135
Época 5000, Pérdida: 0.000124
Época 5200, Pérdida: 0.000108
Época 5400, Pérdida: 0.000096
Época 5600, Pérdida: 0.000355
Época 5800, Pérdida: 0.000077
Época 6000, Pérdida: 0.000073
Época 6200, Pérdida: 0.000234
Época 6400, Pérdida: 0.000059
Época 6600, Pérd

In [ ]:
error_relativo=[]
for i in range(len(training_set)):

  prediccion = red1.predecir(training_set[i])

  res=np.matmul(reduced_basis,salidas[i])-np.matmul(reduced_basis,prediccion[0])
  error_relativo.append(np.linalg.norm(res)/np.linalg.norm(np.matmul(reduced_basis,salidas[i])))

print(error_relativo)
print(np.mean(error_relativo))
print(np.max(error_relativo))

[np.float64(0.005025966466327991), np.float64(0.006750544930895893), np.float64(0.016537056284939445), np.float64(0.0066984582843550265), np.float64(0.0034153630293490083), np.float64(0.0035300437010425518), np.float64(0.003025758181872846), np.float64(0.0016170421947557861), np.float64(0.0057006295439966555), np.float64(0.004419059904316647), np.float64(0.0033752502582755912), np.float64(0.0038771402486974214), np.float64(0.003248201428656482), np.float64(0.004589006568497871), np.float64(0.0062774172892700745), np.float64(0.006157942798266058), np.float64(0.006759616511862895), np.float64(0.00946180277089183), np.float64(0.004477216778262071), np.float64(0.007467420542005011), np.float64(0.012471582955474238), np.float64(0.005449712781485298), np.float64(0.001492765154639193), np.float64(0.001965828170946222), np.float64(0.0013318489973205882), np.float64(0.0023761867392404898), np.float64(0.005886856525485576), np.float64(0.003348475139467758), np.float64(0.011845615762644928), np.f

In [ ]:
error_relativo=[]
for i in range(len(testing_set)):
  prediccion = red1.predecir(testing_set[i])

  res=np.matmul(reduced_basis,salidas_esperadas[i])-np.matmul(reduced_basis,prediccion[0])
  error_relativo.append(np.linalg.norm(res)/np.linalg.norm(np.matmul(reduced_basis,salidas_esperadas[i])))

print(error_relativo)
print(np.mean(error_relativo))
print(np.max(error_relativo))

[np.float64(0.0012581736282881644), np.float64(0.009399799375642354), np.float64(0.009737032141129365), np.float64(0.0017332735526672984), np.float64(0.007795225184832046), np.float64(0.0035888755945578986), np.float64(0.008888342714032708), np.float64(0.0060727652051400585), np.float64(0.001306205322295194), np.float64(0.004730199022337117), np.float64(0.009468414520094623), np.float64(0.008043808904637689), np.float64(0.0032271603615580905), np.float64(0.0010803965160374835), np.float64(0.003069302242753377), np.float64(0.009919708315411184), np.float64(0.0024955585837437085), np.float64(0.005750566027339911), np.float64(0.0063918367928052664), np.float64(0.0037299962737507427)]
0.005384332013952714
0.009919708315411184


# Con rigidez variable

In [7]:
training_set.shape

(150, 12)

In [9]:
reduced_basis.shape

(2692, 16)

In [45]:
red2 = RED_NEURONAL(
    n_entrada=training_set.shape[1],
    m_salida= reduced_basis.shape[1],
    training_set=training_set,
    salidas=salidas,
    funcion_costo=loss_func,
    max_epochs=15000,
    capas_ocultas=3,
    neuronas_por_capa=48,
    lr=0.001,
    tol=1e-10,
    min_delta=1e-10

)

Usando dispositivo: cpu
Forma de X: torch.Size([150, 12])
Forma de Y: torch.Size([150, 16])


In [46]:
red2.entrenar()

Época 0, Pérdida: 0.05178786
Época 200, Pérdida: 0.00287379
Época 400, Pérdida: 0.00103341
Época 600, Pérdida: 0.00055505
Época 800, Pérdida: 0.00037727
Época 1000, Pérdida: 0.00028020
Época 1200, Pérdida: 0.00021831
Época 1400, Pérdida: 0.00018795
Época 1600, Pérdida: 0.00014922
Época 1800, Pérdida: 0.00012591
Época 2000, Pérdida: 0.00010951
Época 2200, Pérdida: 0.00010319
Época 2400, Pérdida: 0.00008715
Época 2600, Pérdida: 0.00009692
Época 2800, Pérdida: 0.00007230
Época 3000, Pérdida: 0.00006576
Época 3200, Pérdida: 0.00005986
Época 3400, Pérdida: 0.00006529
Época 3600, Pérdida: 0.00005004
Época 3800, Pérdida: 0.00004620
Época 4000, Pérdida: 0.00004212
Época 4200, Pérdida: 0.00003855
Época 4400, Pérdida: 0.00003914
Época 4600, Pérdida: 0.00003275
Época 4800, Pérdida: 0.00003086
Época 5000, Pérdida: 0.00003214
Época 5200, Pérdida: 0.00002640
Época 5400, Pérdida: 0.00002692
Época 5600, Pérdida: 0.00015545
Época 5800, Pérdida: 0.00002170
Época 6000, Pérdida: 0.00002045
Época 6200, Pér

In [47]:
error_relativo=[]
for i in range(len(training_set)):

  prediccion = red2.predecir(training_set[i])

  res=np.matmul(reduced_basis,salidas[i])-np.matmul(reduced_basis,prediccion[0])
  error_relativo.append(np.linalg.norm(res)/np.linalg.norm(np.matmul(reduced_basis,salidas[i])))

print(error_relativo)
print(np.mean(error_relativo))
print(np.max(error_relativo))

[np.float64(0.09229527509840833), np.float64(0.04784446994470353), np.float64(0.01227875643329794), np.float64(0.06111739189669707), np.float64(0.0716804996355881), np.float64(0.10535626397624406), np.float64(0.06707719114175922), np.float64(0.017019488058254716), np.float64(0.06710124635587635), np.float64(0.03719744089289741), np.float64(0.026594237029284273), np.float64(0.029061335227206532), np.float64(0.05925885940794454), np.float64(0.03322821051758798), np.float64(0.09461523782339004), np.float64(0.02356331796839447), np.float64(0.04338286924001763), np.float64(0.036472536174340375), np.float64(0.01541497461863555), np.float64(0.07112923758112298), np.float64(0.042291722494746685), np.float64(0.032487740604351334), np.float64(0.04572477441375234), np.float64(0.08433593148099557), np.float64(0.012618861255795413), np.float64(0.07115363298695698), np.float64(0.033829228026326576), np.float64(0.04944680625751388), np.float64(0.03360860560510659), np.float64(0.018495699168294463), n

In [48]:
error_relativo=[]
for i in range(len(testing_set)):
  prediccion = red2.predecir(testing_set[i])

  res=np.matmul(reduced_basis,salidas_esperadas[i])-np.matmul(reduced_basis,prediccion[0])
  error_relativo.append(np.linalg.norm(res)/np.linalg.norm(np.matmul(reduced_basis,salidas_esperadas[i])))

print(error_relativo)
print(np.mean(error_relativo))
print(np.max(error_relativo))

[np.float64(1.1860381746620667), np.float64(3.7330213446419633), np.float64(3.37716438218149), np.float64(4.373785908633264), np.float64(2.5487620810129368), np.float64(6.590922220949718), np.float64(3.891655344480458), np.float64(0.8556176226199926), np.float64(1.4073182728278224), np.float64(0.5305814031913364), np.float64(5.436154022176551), np.float64(10.858677304118334), np.float64(3.9778144823733816), np.float64(2.710391447236741), np.float64(8.895415851487877), np.float64(3.9430019514225116), np.float64(0.9616016315478628), np.float64(1.4810699673878258), np.float64(2.9948626171498662), np.float64(0.7928128647859346)]
3.5273334447443965
10.858677304118334


# Observaciones:

Con una matriz de 150 snapshots no podemos entrenar una red para que prediga una función de $F:\mathbb{R}^{12} → \mathbb{R}^{16}$. Hay que aumentar los snapshots iniciales. Si para aproximar bien una función $F:\mathbb{R}^{3} → \mathbb{R}^{3}$ requerimos 100 snapshots podemos suponer que esta requerira al menos 400 snapshots